In [2]:
import re
import sys
sys.path.append('./readme2kg-exp/src/')
import os
import random
from collections import defaultdict
from termcolor import colored
from functools import partial, reduce
import operator as op
import hashlib
import multiprocessing as mp
import logging

from predictor import BasePredictor, LABELS
from webanno_tsv import webanno_tsv_read_file, Document, Annotation, Token
import utils
import cleaner

In [12]:
phase = 'test_unlabeled'
base_path = f'./readme2kg-exp/data/{phase}'
file_names = [fp for fp in os.listdir(base_path) if os.path.isfile(os.path.join(base_path, fp)) and fp.endswith('.tsv')]
model_name = 'Mistral-7B-Instruct-v0.3'
output_folder = f'./readme2kg-exp/results/{model_name}/{phase}'
os.makedirs(output_folder, exist_ok=True)

In [13]:
prompt_id = 0
prompt_template_path = f'./readme2kg-exp/config/deepseek-chat-prompt-0.txt'
if os.path.isfile(prompt_template_path):
    with open(prompt_template_path, 'r') as fd:
        prompt_template = fd.read()
else:
    prompt_template = ''

# Load Mistral model

In [5]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

'/home/ann/mistral_models/7B-Instruct-v0.3'

In [6]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
model = Transformer.from_folder(mistral_models_path)

# unit test code
sentence_text = """# DejaVu ## Table of Contents =================    * [Code](#code)     * [Install Requirements](#install-requirements)     * [Usage](#usage)     * [Example](#example)   * [Datasets](#datasets)   * [Deployment and Failure Injection Scripts of Train-Ticket](#deployment-and-failure-injection-scripts-of-train-ticket)   * [Citation](#citation)   * [Supplementary details](#supplementary-details)    ## Paper A preprint version: https://arxiv.org/abs/2207.09021 ## Code ### Install 1."""
prompt = prompt_template.replace('{input_text}', sentence_text)

# original code
#prompt = prompt_template.replace('{input_text}', sentence.text)

'''
messages=[
            {"role": "system", "content": "You are a helpful NER annotator"},
            {"role": "user", "content": prompt},
        ]
'''
completion_request = ChatCompletionRequest(messages=[
            {"role": "system", "content": "You are a helpful NER annotator"},
            {"role": "user", "content": prompt},
        ]
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=1000, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens)

print(result)

['```\n# DejaVu\n## Table of Contents =================\n\t* [Code](#code)\n\t* [Install Requirements](#install-requirements)\n\t* [Usage](#usage)\n\t* [Example](#example)\n\t* [Datasets](#datasets)\n\t* [Deployment and Failure Injection Scripts of Train-Ticket](#deployment-and-failure-injection-scripts-of-train-ticket)\n\t* [Citation](#citation)\n\t* [Supplementary details](#supplementary-details)\n\t## Paper\n\tA preprint version: <PUBLICATION>https://arxiv.org/abs/2207.09021</PUBLICATION>\n\t## Code\n\t### Install\n\t1.\n```']


In [7]:
def do_prediction(sentence, tokens, sid_path):
    try:
        #print(f"Process-{os.getpid()} processing {colored(sentence.text, 'red')} ...")
        prompt = prompt_template.replace('{input_text}', sentence.text)

        messages=[
                    {"role": "system", "content": "You are a helpful NER annotator"},
                    {"role": "user", "content": prompt},
                ]
        
        completion_request = ChatCompletionRequest(messages=[
            {"role": "system", "content": "You are a helpful NER annotator"},
            {"role": "user", "content": prompt},
        ])
        tokens = tokenizer.encode_chat_completion(completion_request).tokens

        out_tokens, _ = generate([tokens], model, max_tokens=255, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
        result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
        
        #print(f"Process-{os.getpid()} predict {colored(sentence.text, 'cyan')} successfully")
        with open(sid_path, 'w') as file:
            file.write(result)
    except Exception as ex:
        logging.error(f'[do_prediction] got exception: {ex}')

In [8]:
def extract_annotation_labels_if_possible(predicted_text):
    label_to_text_list = defaultdict(list)
    acc_adjusted_pos = 0
    for label in LABELS:
        regex = f'<{label}>(.*?)</{label}>'
        matches = re.finditer(regex, predicted_text, flags=re.IGNORECASE | re.DOTALL)
        for m in matches:
            adjusted_pos = len(label) + 2
            label_to_text_list[label].append({
                'text': m.group(1),
                'start': m.start(1) - adjusted_pos - acc_adjusted_pos,
                'end': m.end(1) - adjusted_pos - acc_adjusted_pos,
            })
            acc_adjusted_pos += adjusted_pos * 2 + 1
    return label_to_text_list



def post_process(predicted_text, tokens):
    cleaned_text = cleaner.Cleaner(predicted_text).clean()
    label_to_text_list = extract_annotation_labels_if_possible(cleaned_text)
    return label_to_text_list

In [9]:
def predict(sentence, tokens):
    path = f'./readme2kg-exp/results/{model_name}/prompt-{prompt_id}/zzz_{file_name}' # NOTE: prefix zzz for directory sorting, non-sense
    os.makedirs(path, exist_ok=True)
    sid = hashlib.sha256(sentence.text.encode()).hexdigest()[:8]
    #if not os.path.isfile(f'{path}/{sid}.txt'):   # original code
    if os.path.isdir(f'{path}'):
        do_prediction(sentence, tokens, f'{path}/{sid}.txt')

    with open(f'{path}/{sid}.txt', 'r') as fd:
        predicted_text = fd.read()

    label_to_text_list = post_process(predicted_text, tokens)
    # NOTE: sanity checking
    for label, text_list in label_to_text_list.items():
        for text in text_list:
            if text['text'] != sentence.text[text['start']:text['end']]:
                prompt = prompt_template.replace('{input_text}', sentence.text)
                #logging.warning(f"BUG? The predicted text is not exact the same as the original text. \n\nPrompt: {prompt}\nOriginal: {colored(sentence.text, 'green')}\nGenerated: {colored(text['text'], 'red')}\n--------------------------------------------------------------------------------")

    span_tokens_to_label_list = []
    for label, text_list in label_to_text_list.items():
        for text in text_list:
            span_tokens_to_label_list.append({
                'span_tokens': utils.make_span_tokens(tokens, text['start'], text['end']),
                'label': label
            })
    return span_tokens_to_label_list


In [10]:
def call_serial(doc: Document):
    annotations = []
    for sent in doc.sentences:
        tokens = doc.sentence_tokens(sent)
        span_tokens_to_label_list = predict(sentence=sent, tokens=tokens)
        
        # create the annotation instances
        for span_tokens_to_label in span_tokens_to_label_list:
            span_tokens = span_tokens_to_label['span_tokens']
            label = span_tokens_to_label['label']
            if span_tokens is None:
                continue

            annotation = utils.make_annotation(tokens=span_tokens, label=label)
            annotations.append(annotation)

    result = utils.replace_webanno_annotations(doc, annotations=annotations)
    return result

In [14]:
for file_name in file_names:
    file_path = os.path.join(base_path, file_name)
    ref_doc = webanno_tsv_read_file(file_path)
    predicted_doc = call_serial(ref_doc)
    # Verify
    if ref_doc.text != predicted_doc.text:
        #logging.warning('content changed')
        pass
    if len(ref_doc.sentences) == len(predicted_doc.sentences):
        #logging.warning('sentences changed')
        pass
    if len(ref_doc.tokens) == len(predicted_doc.tokens):
        #logging.warning('tokens changed')
        pass
    for s1, s2 in zip(ref_doc.sentences, predicted_doc.sentences):
        if s1 == s2:
            #logging.warning(f'sentence changed, \n{s1}\n{s2}')
            pass

    for t1, t2 in zip(ref_doc.tokens, predicted_doc.tokens):
        if t1 == t2:
            #logging.warning(f'token changed: \n{t1}\n{t2}')
            pass

    logging.warning(f"Predicted {len(predicted_doc.annotations)} annotations")
    prediction_path = os.path.join(output_folder, file_name)
    with open(prediction_path, 'w') as fd:
        fd.write(predicted_doc.tsv())

# Scorer.py

In [4]:
import argparse
import json
import os
from collections import defaultdict
from functools import reduce
from webanno_tsv import webanno_tsv_read_file, Document, Annotation
from typing import List, Union
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

LABELS = [
    'CONFERENCE',
    'DATASET',
    'EVALMETRIC',
    'LICENSE',
    'ONTOLOGY',
    'PROGLANG',
    'PROJECT',
    'PUBLICATION',
    'SOFTWARE',
    'WORKSHOP'
]

def to_char_bio(src_path: str, ref_path: str) -> List[List[str]]:
    ref_doc = webanno_tsv_read_file(ref_path)
    # Parse the WebAnno TSV file
    doc = webanno_tsv_read_file(src_path)
    # Initialize a list to store character-level BIO tags
    bio_tags_list = []
    for target_label in LABELS:
        bio_tags = ['#'] * len(ref_doc.text)  # Default to '#' for all characters
        # Pick interested sentences and default them to 'O'
        for annotation in ref_doc.annotations:
            label = annotation.label
            if label != target_label:
                continue
            sentences = ref_doc.annotation_sentences(annotation)
            for sentence in sentences:
                tokens = ref_doc.sentence_tokens(sentence)
                start_char, end_char = tokens[0].start, tokens[-1].end
                bio_tags[start_char:end_char] = ['O'] * (end_char-start_char)

        for annotation in doc.annotations:
            label = annotation.label
            if label != target_label:
                continue

            start_token, end_token = annotation.tokens[0], annotation.tokens[-1]
            start_char = start_token.start
            end_char = end_token.end
            # Sanity check
            if ref_doc.text[start_char:end_char] != annotation.text:
                msg = f"ERROR: src: {src_path}, annotated '{annotation.text}', text: '{ref_doc.text[start_char:end_char]}'"
                print(msg)

            if 'I-' in bio_tags[start_char]:
                # Overlapping, it's annotated by another annotations, we connect them as one annotations
                pass
            else:
                if bio_tags[start_char] != '#':
                    # Assign BIO tags to characters in the entity span
                    bio_tags[start_char] = f'B-{label}'  # Beginning of the entity

            for i in range(start_char + 1, end_char):
                if bio_tags[i] != '#':
                    bio_tags[i] = f'I-{label}'  # Inside the entity

        # Remove unannotated sentences from bio list.
        bio_tags = [x for x in filter(lambda x: x != '#', bio_tags)]
        bio_tags_list.append(bio_tags)

    return bio_tags_list

In [2]:
def flatten(lst):
    return reduce(lambda x, y: x + y, lst)

In [5]:
import os 

print(os.getcwd())
ref_dir = '../results/Mistral-7B-Instruct-v0.3/test_unlabeled/'
pred_dir = '../results/Mistral-7B-Instruct-v0.3/prompt-0/'
score_dir = '../results/scores/'

os.makedirs(pred_dir, exist_ok=True)
os.makedirs(score_dir, exist_ok=True)

ref_file_names = sorted([fp for fp in os.listdir(ref_dir) if os.path.isfile(f'{ref_dir}/{fp}') and fp.endswith('.tsv')])

if len(ref_file_names) == 0:
    raise Exception("ERROR: No reference files found, configuration error?")

all_ref_bio_tags_list = []
for ref_file_name in ref_file_names:
    src_path = os.path.join(ref_dir, ref_file_name)
    ref_path = src_path
    all_ref_bio_tags_list.append(to_char_bio(src_path, ref_path))

pred_file_names = sorted([fp for fp in os.listdir(pred_dir) if os.path.isfile(f'{pred_dir}/{fp}') and fp.endswith('.tsv')])
all_pred_bio_tags_list = []
for idx, ref_file_name in enumerate(ref_file_names):
    try:
        src_path = os.path.join(pred_dir, ref_file_name)
        ref_path = os.path.join(ref_dir, ref_file_name)
        all_pred_bio_tags_list.append(to_char_bio(src_path, ref_path))
    except FileNotFoundError:
        nbr_labels = len(all_ref_bio_tags_list[idx])
        assert nbr_labels == len(LABELS), "ERROR: reference tags doesn't have ${len(LABELS)} labels."
        pred = []
        for label_idx in range(nbr_labels):
            pred.append(['O'] * len(all_ref_bio_tags_list[idx][label_idx]))

        print(f"WARN: {ref_file_name} is missing, fill 'O' list as default prediction")
        all_pred_bio_tags_list.append(pred)
# Sanity checking
for idx, (ref_list, pred_list) in enumerate(zip(all_ref_bio_tags_list, all_pred_bio_tags_list)):
    for label_idx, (ref, pred) in enumerate(zip(ref_list, pred_list)):
        assert len(ref) == len(pred), f'ERROR: {ref_file_names[idx]}, label: {LABELS[label_idx]}, reference length: {len(ref)}, prediction length: {len(pred)}'

scores = {}
################################################################################
# Consider whole dataset
################################################################################
ref_bio_tags_list = flatten(flatten(all_ref_bio_tags_list))
pred_bio_tags_list = flatten(flatten(all_pred_bio_tags_list))

accuracy = accuracy_score(ref_bio_tags_list, pred_bio_tags_list)
scores['overall_accuracy'] = accuracy
average = 'macro'
ref_bio_tags_list = flatten(flatten(all_ref_bio_tags_list))
pred_bio_tags_list = flatten(flatten(all_pred_bio_tags_list))

f1 = f1_score(ref_bio_tags_list, pred_bio_tags_list, average=average)
precision = precision_score(ref_bio_tags_list, pred_bio_tags_list, average=average)
recall = recall_score(ref_bio_tags_list, pred_bio_tags_list, average=average)
scores[f"overall_{average}_precision"] = precision
scores[f"overall_{average}_recall"] = recall
scores[f"overall_{average}_f1"] = f1


################################################################################
# For each class
################################################################################
label_to_ref_bio_tags_list = defaultdict(list)
label_to_pred_bio_tags_list = defaultdict(list)
for ref_bio_tags_list, pred_bio_tags_list in zip(all_ref_bio_tags_list, all_pred_bio_tags_list):
    if len(ref_bio_tags_list) != len(LABELS):
        print('ERROR: ref bio tags list')
    if len(pred_bio_tags_list) != len(LABELS):
        print('ERROR: pred bio tags list')

    for label, ref_bio_tags, pred_bio_tags in zip(LABELS, ref_bio_tags_list, pred_bio_tags_list):
        label_to_ref_bio_tags_list[label].extend(ref_bio_tags)
        label_to_pred_bio_tags_list[label].extend(pred_bio_tags)
        if len(label_to_ref_bio_tags_list[label]) != len(label_to_pred_bio_tags_list[label]):
            print('ERROR: label_to_ref_pred_bio_tags')


for label in label_to_ref_bio_tags_list.keys():
    ref_bio_tags_list = label_to_ref_bio_tags_list[label]
    pred_bio_tags_list = label_to_pred_bio_tags_list[label]
    accuracy = accuracy_score(ref_bio_tags_list, pred_bio_tags_list)
    f1 = f1_score(ref_bio_tags_list, pred_bio_tags_list, average=average)
    precision = precision_score(ref_bio_tags_list, pred_bio_tags_list, average=average)
    recall = recall_score(ref_bio_tags_list, pred_bio_tags_list, average=average)
    scores[f"{label}_{average}_precision"] = precision
    scores[f"{label}_{average}_recall"] = recall
    scores[f"{label}_{average}_f1"] = f1

print("Scores:\n", json.dumps(scores, indent=2))

with open(os.path.join(score_dir, 'Mistral-7B-Instruct-v0.3-scores.json'), 'w') as fd:
    json.dump(scores, fd, indent=2)

/home/ann/fiz-ddb/notebook/readme2kg-exp/src
WARN: 231sm_Low_Resource_KBP_master_README.md.tsv is missing, fill 'O' list as default prediction
WARN: ARM-software_keyword-transformer_master_README.md.tsv is missing, fill 'O' list as default prediction
WARN: Cardio-AI_3d-mri-domain-adaptation_master_README.md.tsv is missing, fill 'O' list as default prediction
WARN: ChopinSharp_ref-nms_main_README.md.tsv is missing, fill 'O' list as default prediction
WARN: James-Durant_fisher-information_main_README.md.tsv is missing, fill 'O' list as default prediction
WARN: MELALab_nela-gt-2019_master_README.md.tsv is missing, fill 'O' list as default prediction
WARN: allenai_aspire_main_README.md.tsv is missing, fill 'O' list as default prediction
WARN: alpiges_LinConGauss_master_README.md.tsv is missing, fill 'O' list as default prediction
WARN: anonymous-submission-22_dejavu_master_README.md.tsv is missing, fill 'O' list as default prediction
WARN: aspiaspace_earthpt_main_README.md.tsv is missing, 

/home/ann/anaconda3/envs/mathd2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ann/anaconda3/envs/mathd2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ann/anaconda3/envs/mathd2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

Scores:
 {
  "overall_accuracy": 0.9203280609256005,
  "overall_macro_precision": 0.043825145758361925,
  "overall_macro_recall": 0.047619047619047616,
  "overall_macro_f1": 0.045643394636683224,
  "CONFERENCE_macro_precision": 0.3284625952291745,
  "CONFERENCE_macro_recall": 0.3333333333333333,
  "CONFERENCE_macro_f1": 0.3308800402591684,
  "DATASET_macro_precision": 0.3198942498347654,
  "DATASET_macro_recall": 0.3333333333333333,
  "DATASET_macro_f1": 0.3264755480607083,
  "EVALMETRIC_macro_precision": 0.33243162007814847,
  "EVALMETRIC_macro_recall": 0.3333333333333333,
  "EVALMETRIC_macro_f1": 0.3328818660647103,
  "LICENSE_macro_precision": 0.32488479262672815,
  "LICENSE_macro_recall": 0.3333333333333333,
  "LICENSE_macro_f1": 0.3290548424737456,
  "ONTOLOGY_macro_precision": 0.30246913580246915,
  "ONTOLOGY_macro_recall": 0.3333333333333333,
  "ONTOLOGY_macro_f1": 0.31715210355987056,
  "PROGLANG_macro_precision": 0.3069741900054915,
  "PROGLANG_macro_recall": 0.333333333333333

/home/ann/anaconda3/envs/mathd2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ann/anaconda3/envs/mathd2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ann/anaconda3/envs/mathd2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",